In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import time
from datetime import datetime
import numpy as np
import pandas as pd

from acpf import ArchiverConfig

In [2]:
bpl_url = 'http://xf04id-ca1.cs.nsls2.local:17665/mgmt/bpl'
arconf = ArchiverConfig(bpl_url)

# Comparing PVs from Archiver and Channel Finder

Retrieve PVs from Archiver

In [3]:
t1 = datetime.now();
# all_pvs = arconf.getAllPVs(regex = '*Mtr*', limit = 10000)
all_pvs = arconf.getAllPVs(limit = 10000)
t2 = datetime.now();
print('pvs: ', len(all_pvs), "time:", (t2 - t1), " seconds")

pvs:  1972 time: 0:00:00.067603  seconds


In [4]:
# general statistics
systems = arconf.getSystems(all_pvs)
devices = arconf.getDevices(all_pvs)
signals = arconf.getSignals(all_pvs)
print('systems: ', len(set(systems)), 'devices: ', len(set(devices)), 'signals: ', len(set(signals)))

systems:  24 devices:  135 signals:  412


Select new PVs from Channel Finder sources

In [5]:
bl = "xf04"
t1 = datetime.now();
new_pvs = arconf.selectNewPVs(bl, system = '', device = '.*', signal = '.*Mtr.*StringOut_')
t2 = datetime.now();
print('pvs: ', len(new_pvs), "time:", (t2 - t1), " seconds")

pvs:  8512 time: 0:00:00.216985  seconds


In [6]:
# general statistics
systems = arconf.getSystems(new_pvs)
devices = arconf.getDevices(new_pvs)
signals = arconf.getSignals(new_pvs)
print('systems: ', len(set(systems)), 'devices: ', len(set(devices)), 'signals: ', len(set(signals)))

systems:  12 devices:  266 signals:  32


Find the difference betweem the Archiver and Channel Finder collections of PVs

In [7]:
diff_pvs = list(set(new_pvs) - set(all_pvs))
len(diff_pvs)

8510

# Archiving PVs

Select 1000 PVs for archiving

In [8]:
demo_pvs = diff_pvs[0:1000]

Send requests for archiving PVs

In [9]:
t1 = datetime.now();
archive_pvs = arconf.archivePV(demo_pvs)
t2 = datetime.now();
print("time:", (t2 - t1), " seconds")

time: 0:00:20.674778  seconds


Wait 5 minutes

In [10]:
time.sleep(300) 

Check status of PVs

In [11]:
t1 = datetime.now();
archived, others = arconf.getArchivingStatus(demo_pvs)
t2 = datetime.now();
print('archived: ', len(archived), 'others:', len(others), "time:", (t2 - t1), " seconds")

archived:  434 others: 566 time: 0:00:01.155144  seconds


# Restoring the original configuration

Abort requests for unarchived PVs

In [12]:
t1 = datetime.now();
for i, pv in enumerate(others, start=1):
    arconf.abortArchivingPV(pv)
t2 = datetime.now();    
never_pvs = arconf.getNeverConnectedPVs()
print('never connected pvs: ', len(never_pvs), "time:", (t2 - t1), " seconds")

never connected pvs:  0 time: 0:00:16.633981  seconds


Pause archived PVs

In [13]:
paused_pvs_1 = arconf.getPausedPVsReport()
t1 = datetime.now();
pause_pvs = arconf.pauseArchivingPV(archived)
t2 = datetime.now(); 
time.sleep(30)
paused_pvs_2= arconf.getPausedPVsReport()
print('paused pvs, before: ', len(paused_pvs_1), 'after :', len(paused_pvs_2), "time:", (t2 - t1), " seconds")

paused pvs, before:  225 after : 659 time: 0:00:00.340455  seconds


Delete paused PVs

In [14]:
t1 = datetime.now();
for i, pv in enumerate(archived):
    delete_pv = arconf.deletePV(pv)
t2 = datetime.now();
paused_pvs = arconf.getPausedPVsReport()
print('paused pvs: ', len(paused_pvs), "time:", (t2 - t1), " seconds")

paused pvs:  225 time: 0:00:04.161084  seconds


## Check the Archiver configuration

In [15]:
final_pvs = arconf.getAllPVs(limit = 10000)
print(len(final_pvs))

1972
